# Demo File
This demo tests the functionality of the EchoStateNetwork with different time series

In [8]:
import matplotlib.pyplot as plt
import numpy as np

from ESN import EchoStateNetwork
from ESN import mackey_glass
from ESN import GinfActivator

ginf_activator = GinfActivator(V_min=-5, V_max=5, num_points=1000)

# Steady State Conductance Activator

In [ ]:
lookup_table = ginf_activator.get_lookup_table()

plt.figure(figsize=(8, 6))
plt.plot(lookup_table[0], lookup_table[1], label=r"$g_{\text{inf}}(V)$", color="blue", linewidth=2)
plt.xlabel(r"$V$", fontsize=14)
plt.ylabel(r"$g_{\text{inf}}(V)$", fontsize=14)
plt.title(r"$g_{\text{inf}}(V) = \int_0^L g(x, V) \, dx$", fontsize=16)
plt.grid(True, linestyle="--", alpha=0.6)
plt.legend(fontsize=12)
plt.savefig("output/ginf_activator_plot.pdf")
plt.savefig("output/ginf_activator_plot.png")

## ESN with Mackey-Glass time series

In [10]:
esn = EchoStateNetwork(
    input_dim=1,
    reservoir_size=200,
    output_dim=1,
    leaking_rate=1,
    step_size=0.3,
    time_scale=1,
    spectral_radius=0.9,
    sparsity=0.85,
    input_scaling=0.5,
    regularization=1e-6,
    activation=ginf_activator.activate,
)

In [11]:
# Generate Mackey-Glass time series
mg_series = mackey_glass(tau=17, n=10000, n_samples=5000)

# Normalize data between -1 and 1
mg_series = 2 * (mg_series - mg_series.min()) / (mg_series.max() - mg_series.min()) - 1

# Create input/output pairs for time series prediction
inputs = mg_series[:-1].reshape(-1, 1)
targets = mg_series[1:].reshape(-1, 1)

# Split into train and test sets
train_len = 4000
test_len = 1000
train_inputs = inputs[:train_len]
train_targets = targets[:train_len]
test_inputs = inputs[train_len : train_len + test_len]
test_targets = targets[train_len : train_len + test_len]

In [12]:
# Train on first part of series
esn.fit(train_inputs, train_targets, washout=200)

# Generate predictions on test set
predictions = esn.predict(test_inputs)

In [ ]:
# Calculate prediction error
mse = np.mean((predictions - test_targets) ** 2)
print(f"Test MSE: {mse:.5f}")

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(test_targets[:200], label="True MG series", alpha=0.8)
plt.plot(predictions[:200], label="ESN predictions", linestyle="--", alpha=0.8)
plt.title("Mackey-Glass Time Series Prediction (First 200 Steps)")
plt.xlabel("Time Step")
plt.ylabel("Value")
plt.legend()
plt.savefig("output/mg_prediction_plot.pdf")
plt.savefig("output/mg_prediction_plot.png")

In [ ]:
# Plot phase space reconstruction
plt.figure(figsize=(8, 6))
plt.plot(test_targets[:-1], test_targets[1:], "b-", alpha=0.5, label="True")
plt.plot(predictions[:-1], predictions[1:], "r--", alpha=0.5, label="Predicted")
plt.title("Phase Space Reconstruction")
plt.xlabel("x(t)")
plt.ylabel("x(t+1)")
plt.legend()
plt.savefig("output/mg_phase_space_plot.pdf")
plt.savefig("output/mg_phase_space_plot.png")

## ESN with Sine Wave time series

In [16]:
# Initialize and train ESN
esn = EchoStateNetwork(
    input_dim=1,
    reservoir_size=100,
    output_dim=1,
    leaking_rate=1,
    step_size=0.3,
    time_scale=1,
    spectral_radius=0.95,
    sparsity=0.7,
    input_scaling=1.0,
    regularization=1e-4,
    activation=np.tanh,
)

In [17]:
# Generate synthetic data (simple sine wave prediction)
t = np.linspace(0, 6 * np.pi, 1000)
data = np.sin(t) * np.cos(t) * t

# Create input/output pairs for time series prediction
inputs = data[:-1].reshape(-1, 1)
targets = data[1:].reshape(-1, 1)


In [18]:
# Train on synthetic data
esn.fit(inputs, targets, washout=100)

# Generate predictions
predictions = esn.predict(inputs)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(t[:-1], targets, label="True signal")
plt.plot(t[:-1], predictions, label="ESN prediction", linestyle="--")
plt.title("ESN Time Series Prediction")
plt.xlabel("Time")
plt.ylabel("Value")
plt.legend()
plt.savefig("output/sine_prediction_plot.pdf")
plt.savefig("output/sine_prediction_plot.png")

# Optimize for MG Data

In [20]:
import optuna
import tensorflow.keras.activations as activations

def clip(x):
    return np.clip(x, 0, 1)

def leaky_relu(x, alpha=0.01):
    return np.clip(np.where(x > 0, x, alpha * x), 0, 1)

activation_functions = {
    "tanh": activations.tanh,
    "clip": clip,
    "leaky_relu": leaky_relu,
    "sigmoid": activations.sigmoid,
    "ginf": ginf_activator.activate
}

In [21]:
def objective(trial):
    mg_series = mackey_glass(tau=17, n=10000, n_samples=5000)
    mg_series = 2 * (mg_series - mg_series.min()) / (mg_series.max() - mg_series.min()) - 1

    inputs = mg_series[:-1].reshape(-1, 1)
    targets = mg_series[1:].reshape(-1, 1)

    train_len = 4000
    test_len = 1000
    train_inputs = inputs[:train_len]
    train_targets = targets[:train_len]
    test_inputs = inputs[train_len : train_len + test_len]
    test_targets = targets[train_len : train_len + test_len]

    reservoir_size = trial.suggest_int("reservoir_size", 50, 500)
    leaking_rate = trial.suggest_float("leaking_rate", 0.1, 1.0)
    step_size = trial.suggest_float("step_size", 0.1, 1.0)
    time_scale = trial.suggest_float("time_scale", 0.5, 2.0)
    spectral_radius = trial.suggest_float("spectral_radius", 0.1, 1.5)
    sparsity = trial.suggest_float("sparsity", 0.1, 0.9)
    input_scaling = trial.suggest_float("input_scaling", 0.1, 2.0)
    regularization = trial.suggest_float("regularization", 1e-6, 1e-2, log=True)

    activation_name = trial.suggest_categorical(
        "activation", ["tanh", "clip", "leaky_relu", "sigmoid", "ginf"]
    )
    activation_func = activation_functions[
        activation_name
    ]

    esn = EchoStateNetwork(
        input_dim=1,
        reservoir_size=reservoir_size,
        output_dim=1,
        leaking_rate=leaking_rate,
        step_size=step_size,
        time_scale=time_scale,
        spectral_radius=spectral_radius,
        sparsity=sparsity,
        input_scaling=input_scaling,
        regularization=regularization,
        activation=activation_func,
    )
    esn.fit(train_inputs, train_targets, washout=200)
    predictions = esn.predict(test_inputs)
    mse = np.mean((predictions - test_targets) ** 2)
    if mse > 10:
        return 10
    return mse

In [ ]:
!del optuna_esn.db

study = optuna.create_study(
    direction="minimize",
    study_name="esn_optimization_MG",
    storage="sqlite:///optuna_esn_MG.db",
    load_if_exists=True,
)
study.optimize(objective, n_trials=500, n_jobs=-1)

best_params = study.best_params
print("Best Hyperparameters:", best_params)

optuna-dashboard sqlite:///optuna_esn.db